Initialization  

Just importing necessary packages and connecting to the database. 

In [2]:
from telnetlib import STATUS
import psycopg2
import pandas as pd
import sys
from tqdm import tqdm

password = 'PAiac14-'
host = 'greenplum01.corral.tacc.utexas.edu'
port = 5432
database = 'uthealth'

username = 'piacobelli'
connection = psycopg2.connect(
    host = host, 
    database = database, 
    user = username, 
    port = port, 
    password = password, 
    keepalives = 1, keepalives_idle = 100
)
connection.autocommit = True


C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Row Count And Claim Count

Creating an initial table to compare the row counts, member ID counts, and claim ID counts between the raw data and the data warehouse. 

In [6]:
tableCreateQuery = ''' 
DROP TABLE IF EXISTS qa_reporting.dw_iqva_claim_detail_counts;
CREATE TABLE qa_reporting.dw_iqva_claim_detail_counts
(
    calendar_year INT,
    table_src TEXT,
    dw_row_count BIGINT,
    src_row_count BIGINT,
    row_count_diff BIGINT,
    row_count_diff_percentage FLOAT,
    dw_uth_clm_id_count BIGINT,
    dw_src_clm_id_count BIGINT,
    src_clm_count BIGINT,
    clm_count_diff BIGINT,
    clm_count_percentage FLOAT,
    dw_uth_mbr_id_count BIGINT,
    dw_src_mbr_id_count BIGINT,
    src_mbr_count BIGINT,
    mbr_count_diff BIGINT,
    mbr_count_percentage FLOAT,
    date_generated DATE
);
'''

with connection.cursor() as cursor:
    cursor.execute(tableCreateQuery)

In [7]:
with connection.cursor() as cursor:
    countInsertQuery = f'''
        INSERT INTO qa_reporting.dw_iqva_claim_detail_counts
        (
            calendar_year, 
            table_src, 
            dw_row_count, 
            date_generated
        )
        SELECT 
            year, 
            table_id_src, 
            COUNT(*), 
            current_date
        FROM dw_staging.iqva_claim_detail
        GROUP BY 1,2;
    '''

    cursor.execute(countInsertQuery)

In [8]:
with connection.cursor() as cursor:

    clmCountUpdateQuery = f'''
        UPDATE qa_reporting.dw_iqva_claim_detail_counts b
        SET dw_uth_clm_id_count = count
        FROM (
            SELECT 
                year, 
                COUNT(DISTINCT uth_claim_id) AS count 
            FROM dw_staging.iqva_claim_detail
            GROUP BY 1
        ) a
        WHERE a.year = b.calendar_year
    '''

    cursor.execute(clmCountUpdateQuery)

In [10]:
with connection.cursor() as cursor:

    mbrCountUpdateQuery = f'''
        UPDATE qa_reporting.dw_iqva_claim_detail_counts b
        SET dw_uth_mbr_id_count = count
        FROM (
            SELECT 
                year, 
                COUNT(DISTINCT uth_member_id) AS count 
            FROM dw_staging.iqva_claim_detail
            GROUP BY 1
        ) a
        WHERE a.year = b.calendar_year
    '''

    cursor.execute(mbrCountUpdateQuery)

In [9]:
with connection.cursor() as cursor:

    srcCountUpdateQuery = f'''
        UPDATE qa_reporting.dw_iqva_claim_detail_counts b
        SET dw_src_clm_id_count = clm_count, dw_src_mbr_id_count = mbr_count
        FROM (
            SELECT 
                year, 
                COUNT(DISTINCT claim_id_src) AS clm_count, 
                COUNT(DISTINCT member_id_src) AS mbr_count 
            FROM dw_staging.iqva_claim_detail
            GROUP BY 1
        ) a
        WHERE a.year = b.calendar_year
    '''

    cursor.execute(srcCountUpdateQuery)

In [4]:
with connection.cursor() as cursor:

    compUpdateQuery = '''    
        WITH clm_counts AS (
            SELECT 
                year,
                COUNT(*) row_count, 
                COUNT(DISTINCT pat_id) pat_count, 
                COUNT(DISTINCT derv_claimno) clm_count
            FROM dev.pi_iqvia_derv_claimno_new_all_yr
            WHERE new_rectype != 'P' OR new_rectype IS NULL
            GROUP BY 1
        )
        UPDATE qa_reporting.dw_iqva_claim_detail_counts a
        SET src_row_count = b.row_count,
            row_count_diff = dw_row_count - b.row_count,
            row_count_diff_percentage = 100. * abs( dw_row_count - b.row_count) / b.row_count,
            src_clm_count = clm_count,
            clm_count_diff = dw_uth_clm_id_count - b.clm_count,
            clm_count_percentage = 100. * abs(dw_uth_clm_id_count - b.clm_count) / b.clm_count,
            src_mbr_count = b.pat_count,
            mbr_count_diff = dw_uth_mbr_id_count - b.pat_count,
            mbr_count_percentage = 100. * abs(dw_uth_mbr_id_count - b.pat_count) / b.pat_count
        FROM clm_counts b
        WHERE a.calendar_year = b.year;
    '''

    cursor.execute(compUpdateQuery)

    

In [15]:
countsDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_claim_detail_counts;', con=connection)
countsDf.sort_values('calendar_year')
print(countsDf)

    calendar_year table_src  dw_row_count  src_row_count  row_count_diff  \
0            2018    claims     329711260      329711260               0   
1            2020    claims     262362411      262362411               0   
2            2008    claims     752001864      752001864               0   
3            2021    claims     295469952      295469952               0   
4            2014    claims     615792857      615792857               0   
5            2009    claims     759139403      759139403               0   
6            2019    claims     333397785      333397785               0   
7            2006    claims     547972991      547972991               0   
8            2012    claims     615799526      615799526               0   
9            2011    claims     651936894      651936894               0   
10           2017    claims     465448852      465448852               0   
11           2013    claims     575391138      575391138               0   
12          

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_5068\204843148.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  countsDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_claim_detail_counts;', con=connection)


In [16]:
countsDf.sort_values('row_count_diff_percentage', ascending=False)[['calendar_year', 'table_src', 'row_count_diff_percentage', 'clm_count_percentage', 'mbr_count_percentage']]

,calendar_year,table_src,row_count_diff_percentage,clm_count_percentage,mbr_count_percentage
0,2018,claims,0.0,0.0,0.0
1,2020,claims,0.0,0.0,0.0
16,2007,claims,0.0,0.0,0.0
15,2015,claims,0.0,0.0,0.0
14,2023,claims,0.0,0.0,0.0
13,2022,claims,0.0,0.0,0.0
12,2016,claims,0.0,0.0,0.0
11,2013,claims,0.0,0.0,0.0
10,2017,claims,0.0,0.0,0.0
9,2011,claims,0.0,0.0,0.0


Place of Service

Creating a table to count different place of service codes. 

In [17]:
posQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_pos;
    SELECT 
        year, 
        place_of_service, 
        COUNT(*)
    INTO qa_reporting.dw_iqva_pos
    FROM dw_staging.iqva_claim_detail
    GROUP BY 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(posQuery)

In [18]:
posDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_pos;', con=connection)
print(posDf)

     year place_of_service      count
0    2017               22   97427199
1    2015               24    5564538
2    2009               25       1348
3    2017               25      13701
4    2018               65    2532643
..    ...              ...        ...
775  2014               25      12793
776  2018               11  150507119
777  2010               11  309969651
778  2015               07       6112
779  2016               81   56393103

[780 rows x 3 columns]


C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_5068\208129925.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  posDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_pos;', con=connection)


In [19]:
posDf['place_of_service'].unique()

array(['22', '24', '25', '65', '07', '12', '21', '99', '52', '19', '15',
       '17', '53', 'ZZ', '33', '11', '08', '05', '06', '04', '20', '16',
       '71', '01', '42', '61', '49', '60', '55', '02', '62', '13', '32',
       '23', '81', '34', '26', '50', '31', '54', '57', '41', '56', '72',
       '18', '51', '10', '03', '14'], dtype=object)

In [21]:
posAntiJoinQuery = '''
    SELECT a.* 
    FROM qa_reporting.dw_iqva_pos a
    LEFT JOIN reference_tables.ref_place_of_service b
    ON a.place_of_service = b.place_of_treatment_cd
    WHERE b.place_of_treatment_cd IS NULL;
;
'''

posDf = pd.read_sql(posAntiJoinQuery, con=connection)
posDf.sort_values(['year', 'place_of_service'])
print(posDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_5068\3414682347.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  posDf = pd.read_sql(posAntiJoinQuery, con=connection)


    year place_of_service     count
0   2021               ZZ   6773067
1   2022               ZZ   4568820
2   2018               ZZ   7152332
3   2006               ZZ  26549465
4   2012               ZZ  24411860
5   2014               ZZ  13642692
6   2008               ZZ  35303171
7   2015               ZZ  13822885
8   2007               ZZ  29469569
9   2016               ZZ  12784943
10  2011               ZZ  25390704
11  2020               ZZ   6275652
12  2009               ZZ  37169096
13  2023               ZZ    426553
14  2019               ZZ   6020768
15  2013               ZZ  16097126
16  2017               ZZ   9525594
17  2010               ZZ  19596902


Revenue Code

Creating a table to count different revenue codes.

In [22]:
revQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_revenue_cd;

    SELECT 
        year, 
        revenue_cd, 
        count(*)
    INTO qa_reporting.dw_iqva_revenue_cd
    FROM dw_staging.iqva_claim_detail
    GROUP BY 1,2
;'''

with connection.cursor() as cursor:
    cursor.execute(revQuery)

In [23]:
revAntiJoinQuery = '''
    SELECT a.* 
    FROM qa_reporting.dw_iqva_revenue_cd a
    LEFT JOIN reference_tables.ref_revenue_code b ON LPAD(a.revenue_cd, 4, '0') = b.revenue_cd
    WHERE b.revenue_cd IS NULL AND a.revenue_cd IS NOT NULL;
'''
invalidRevDf = pd.read_sql(revAntiJoinQuery, con=connection)
invalidRevDf.sort_values(['year', 'revenue_cd'])
print(invalidRevDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_5068\3586865327.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  invalidRevDf = pd.read_sql(revAntiJoinQuery, con=connection)


Empty DataFrame
Columns: [year, revenue_cd, count]
Index: []


In [24]:
pd.DataFrame(invalidRevDf['revenue_cd'].unique())

,0


In [25]:
sumInvalidRevsDf = invalidRevDf.groupby('year').sum()
print(sumInvalidRevsDf)

Empty DataFrame
Columns: [revenue_cd, count]
Index: []


In [26]:
revDf = pd.read_sql('SELECT year, SUM(count) FROM qa_reporting.dw_iqva_revenue_cd GROUP BY 1', con=connection)
revDf['sum'] = revDf['sum'].astype(int)
revDf = revDf.set_index('year')
print(revDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_5068\2727154628.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  revDf = pd.read_sql('SELECT year, SUM(count) FROM qa_reporting.dw_iqva_revenue_cd GROUP BY 1', con=connection)


            sum
year           
2020  262362411
2008  752001864
2021  295469952
2018  329711260
2014  615792857
2009  759139403
2019  333397785
2017  465448852
2013  575391138
2016  603322301
2023  104947333
2015  582695183
2007  662430963
2010  651457191
2006  547972991
2011  651936894
2012  615799526
2022  296642507


In [36]:
100. * (sumInvalidRevsDf['count'] if sumInvalidRevsDf.shape[0] > 0  else 0) / revDf['sum']

year
2020    0.0
2008    0.0
2021    0.0
2018    0.0
2014    0.0
2009    0.0
2019    0.0
2017    0.0
2013    0.0
2016    0.0
2023    0.0
2015    0.0
2007    0.0
2010    0.0
2006    0.0
2011    0.0
2012    0.0
2022    0.0
Name: sum, dtype: float64

Bill Type Code

Creating a table to count different bill type codes. 

In [28]:
billQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_bill_cd;

    SELECT 
        year, 
        bill_type_inst || bill_type_class || bill_type_freq as bill_cd, 
        COUNT(*)
    INTO qa_reporting.dw_iqva_bill_cd
    FROM dw_staging.iqva_claim_detail
    GROUP BY 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(billQuery)

In [29]:
billAntiJoinQuery = '''
    SELECT a.* 
    FROM qa_reporting.dw_iqva_bill_cd a
    LEFT JOIN reference_tables.ref_bill_type_cd b ON a.bill_cd = b.bill_type_cd
    WHERE b.bill_type_cd IS NULL AND a.bill_cd IS NOT NULL;
'''
invalidBillDf = pd.read_sql(billAntiJoinQuery, con=connection)
invalidBillDf.sort_values(['year', 'bill_cd'])
print(invalidBillDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_5068\2562219315.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  invalidBillDf = pd.read_sql(billAntiJoinQuery, con=connection)


Empty DataFrame
Columns: [year, bill_cd, count]
Index: []


In [30]:
pd.DataFrame(invalidBillDf['bill_cd'].unique())

,0


In [32]:
sumInvalidBillsDf = invalidBillDf.groupby('year').sum()
print(sumInvalidBillsDf)

Empty DataFrame
Columns: [bill_cd, count]
Index: []


In [33]:
billDf = pd.read_sql('SELECT year, SUM(count) FROM qa_reporting.dw_iqva_bill_cd group by 1', con=connection)
billDf['sum'] = billDf['sum'].astype(int)
billDf = billDf.set_index('year')
print(billDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_5068\2581626132.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  billDf = pd.read_sql('SELECT year, SUM(count) FROM qa_reporting.dw_iqva_bill_cd group by 1', con=connection)


            sum
year           
2018  329711260
2020  262362411
2008  752001864
2021  295469952
2014  615792857
2009  759139403
2019  333397785
2006  547972991
2011  651936894
2012  615799526
2022  296642507
2017  465448852
2013  575391138
2016  603322301
2023  104947333
2015  582695183
2007  662430963
2010  651457191


In [34]:
100. * (sumInvalidBillsDf['count'] if sumInvalidBillsDf.shape[0] > 0  else 0) / billDf['sum']

year
2018    0.0
2020    0.0
2008    0.0
2021    0.0
2014    0.0
2009    0.0
2019    0.0
2006    0.0
2011    0.0
2012    0.0
2022    0.0
2017    0.0
2013    0.0
2016    0.0
2023    0.0
2015    0.0
2007    0.0
2010    0.0
Name: sum, dtype: float64

CPT HCPCS Codes  

Creating a table to count different CPT HCPCS codes.

In [7]:
procQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_cpt_proc_counts;
   
    SELECT 
        year, 
        cpt_hcpcs_cd, 
        COUNT(*) AS proc_count
    INTO qa_reporting.dw_iqva_cpt_proc_counts
    FROM dw_staging.iqva_claim_detail
    GROUP BY 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(procQuery)

In [8]:
procDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_cpt_proc_counts;', con=connection)
print(procDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_12816\2574447171.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  procDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_cpt_proc_counts;', con=connection)


        year cpt_hcpcs_cd  proc_count
0       2011        J1364         324
1       2015        L3764          70
2       2018        27306          98
3       2022        96375      595290
4       2020        44626         502
...      ...          ...         ...
336296  2018        D1550          64
336297  2021        28312         152
336298  2014        V5267        1355
336299  2011        74220       59795
336300  2018        2AHLS           2

[336301 rows x 3 columns]


In [13]:
procAntiJoinQuery = '''
    SELECT a.*
    FROM qa_reporting.dw_iqva_cpt_proc_counts a
    LEFT JOIN (SELECT DISTINCT procedure_cd FROM iqvia.pr_lookup) b ON a.cpt_hcpcs_cd = b.procedure_cd
    WHERE b.procedure_cd IS NULL AND a.cpt_hcpcs_cd IS NOT NULL
;
'''

invalidProcDf = pd.read_sql(procAntiJoinQuery, con=connection)
print(invalidProcDf)


C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_12816\2528136225.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  invalidProcDf = pd.read_sql(procAntiJoinQuery, con=connection)


       year cpt_hcpcs_cd  proc_count
0      2015        35025           2
1      2021        ZDB52          12
2      2009        A0452           1
3      2011        ZDB84           4
4      2014        1AFPS          29
...     ...          ...         ...
57960  2007        A1270           1
57961  2007        HOSPT           1
57962  2012        3BGL1           2
57963  2010        U0914           3
57964  2006          063           1

[57965 rows x 3 columns]


In [14]:
invalidProcDf.groupby('year')['proc_count'].sum()

year
2006    3179714
2007    2880799
2008    2409859
2009    2285684
2010    1720424
2011    1876888
2012    1511091
2013    1256002
2014    3028539
2015    5050726
2016    5444778
2017    5474327
2018    4915982
2019    4300412
2020    1054867
2021     138666
2022     138645
2023      34432
Name: proc_count, dtype: int64

In [15]:
procCompDf = pd.DataFrame({'overall_proc_count': procDf.groupby('year')['proc_count'].sum(),
                            'invalid_proc_count': invalidProcDf.groupby('year')['proc_count'].sum(),
                            'valid_proc_count': procDf.groupby('year')['proc_count'].sum() - invalidProcDf.groupby('year')['proc_count'].sum()})
procCompDf.loc[procCompDf['valid_proc_count'].isna(),'valid_proc_count'] = procCompDf.loc[procCompDf['valid_proc_count'].isna(),'overall_proc_count']
procCompDf['valid_proc_count'] =  procCompDf['valid_proc_count'].astype(int)
procCompDf['invalid_to_valid_percent'] = 100. * procCompDf['invalid_proc_count'] / procCompDf['valid_proc_count']
procCompDf 

,overall_proc_count,invalid_proc_count,valid_proc_count,invalid_to_valid_percent
year,,,,
2006,547972991,3179714,544793277,0.583655
2007,662430963,2880799,659550164,0.436782
2008,752001864,2409859,749592005,0.321489
2009,759139403,2285684,756853719,0.301998
2010,651457191,1720424,649736767,0.264788
2011,651936894,1876888,650060006,0.288725
2012,615799526,1511091,614288435,0.245990
2013,575391138,1256002,574135136,0.218764
2014,615792857,3028539,612764318,0.494242


Its hard to check the accuracy of cpt-hcpcs codes right now because the reference tables don't fully correspond with the Iqvia data. 

Discharge Status

Getting discharge status.

In [52]:
discQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_discharge_counts;
    SELECT 
        year, 
        discharge_status, 
        COUNT(*) as count
    INTO qa_reporting.dw_iqva_discharge_counts
    FROM dw_staging.iqva_claim_detail
    GROUP BY 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(discQuery)

In [53]:
discDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_discharge_counts;', con=connection)
print(discDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_5068\970270768.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  discDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_discharge_counts;', con=connection)


      year discharge_status      count
0     2012             None  516326720
1     2016               50     165785
2     2011               35          1
3     2023               81       2413
4     2021               51     168133
...    ...              ...        ...
1089  2015               81      13144
1090  2006               34          1
1091  2017               02    1124880
1092  2019               21       5556
1093  2008               22        312

[1094 rows x 3 columns]


In [60]:
discDf['discharge_status'].unique()

array(['50', '35', '81', '51', '90', '85', '03', '72', '12', '18', '43',
       '06', '95', '93', '92', '21', '07', '71', '70', '86', '97', '61',
       '10', '65', '28', '04', '99', '52', '59', '09', '64', '32', '76',
       '60', '83', '38', '19', '08', '55', '94', '82', '1 ', '66', '98',
       '13', '11', '84', '89', '8 ', '30', '91', '57', '69', '16', '01',
       '80', '00', '22', '63', '87', '26', '24', '3 ', '47', '62', '74',
       '15', '05', '36', '34', '88', '31', '73', '56', '17', '48', '0 ',
       '33', '54', '45', '02', '96', '23', '39', '2 ', '7 ', '78', '14',
       '44', '25', '6 ', '9 ', '27', '37', '46', '77', '79', '53', '29',
       '68', '5 ', '67', '49'], dtype=object)

In [54]:
discDf[discDf['discharge_status'].isna()]

,year,discharge_status,count
0,2012,None,516326720
25,2010,None,563601109
95,2023,None,74051430
98,2008,None,668603464
103,2022,None,210410258
159,2015,None,471563640
249,2013,None,472030995
265,2009,None,675085808
333,2017,None,370095588
400,2011,None,560590013


In [55]:
discDf = discDf[~discDf['discharge_status'].isna()]
print(discDf)

      year discharge_status    count
1     2016               50   165785
2     2011               35        1
3     2023               81     2413
4     2021               51   168133
5     2019               90     2526
...    ...              ...      ...
1089  2015               81    13144
1090  2006               34        1
1091  2017               02  1124880
1092  2019               21     5556
1093  2008               22      312

[1076 rows x 3 columns]


In [61]:
discAntiJoinQuery = '''
    select a.*
from qa_reporting.dw_iqva_discharge_counts a
left join reference_tables.ref_discharge_status b
on a.discharge_status = b.discharge_status
where b.discharge_status is null
and a.discharge_status is not null
;
'''

invalidDiscDf = pd.read_sql(discAntiJoinQuery, con=connection)
print(invalidDiscDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_5068\3822852335.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  invalidDiscDf = pd.read_sql(discAntiJoinQuery, con=connection)


     year discharge_status  count
0    2014               18    303
1    2016               74    157
2    2014               36      5
3    2013               39     44
4    2010               23    321
..    ...              ...    ...
487  2019               16     21
488  2014               80     20
489  2011               19    193
490  2006               12    126
491  2022               99     89

[492 rows x 3 columns]


In [62]:
discCompDf = pd.DataFrame(
    {
        'overall_count': discDf.groupby('year')['count'].sum(),
        'invalid_count': invalidDiscDf.groupby('year')['count'].sum(),
        'valid_count': discDf.groupby('year')['count'].sum() - invalidDiscDf.groupby('year')['count'].sum()
    }
)
discCompDf.loc[discCompDf['valid_count'].isna(),'valid_count'] = discCompDf.loc[discCompDf['valid_count'].isna(),'overall_count']
discCompDf['valid_count'] =  discCompDf['valid_count'].astype(int)
discCompDf['invalid_to_valid_percent'] = 100. * discCompDf['invalid_count'] / discCompDf['valid_count']
print(discCompDf) 

      overall_count  invalid_count  valid_count  invalid_to_valid_percent
year                                                                     
2006       46060057          27767     46032290                  0.060321
2007       53448204          75530     53372674                  0.141514
2008       83398400          96065     83302335                  0.115321
2009       84053595         100597     83952998                  0.119825
2010       87856082          94409     87761673                  0.107574
2011       91346881         108416     91238465                  0.118827
2012       99472806          51577     99421229                  0.051877
2013      103360143          27334    103332809                  0.026452
2014      120575513          10418    120565095                  0.008641
2015      111131543          10936    111120607                  0.009842
2016      120102773           8638    120094135                  0.007193
2017       95353264           2079    

DRG Code

Creating a table to count different DRG codes. 

In [59]:
query = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_drg_counts;

    SELECT 
        year, 
        drg_cd, 
        COUNT(*) as drg_count
    INTO qa_reporting.dw_iqva_drg_counts
    FROM dw_staging.iqva_claim_detail
    GROUP BY 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [63]:
drgDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_drg_counts;', con=connection)
print(drgDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_5068\845894363.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  drgDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_drg_counts;', con=connection)


    year drg_cd  drg_count
0   2018   None  329711260
1   2017   None  465448852
2   2023   None  104947333
3   2016   None  603322301
4   2009   None  759139403
5   2014   None  615792857
6   2015   None  582695183
7   2013   None  575391138
8   2007   None  662430963
9   2008   None  752001864
10  2011   None  651936894
11  2019   None  333397785
12  2006   None  547972991
13  2010   None  651457191
14  2020   None  262362411
15  2022   None  296642507
16  2012   None  615799526
17  2021   None  295469952


In [64]:
drgDf['drg_cd'].unique()

array([None], dtype=object)

In [65]:
drgDf[drgDf['drg_cd'].isna()]

,year,drg_cd,drg_count
0,2018,None,329711260
1,2017,None,465448852
2,2023,None,104947333
3,2016,None,603322301
4,2009,None,759139403
5,2014,None,615792857
6,2015,None,582695183
7,2013,None,575391138
8,2007,None,662430963
9,2008,None,752001864


Provider Type

Creating a table to count different provider types. 

In [67]:
provQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_provider_type_counts;

    SELECT 
        year, 
        provider_type, 
        COUNT(*) as type_count
    INTO qa_reporting.dw_iqva_provider_type_counts
    FROM dw_staging.iqva_claim_detail
    GROUP BY 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(provQuery)

In [68]:
provDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_provider_type_counts;', con=connection)
print(provDf)

    year provider_type  type_count
0   2019          None   333397785
1   2023          None   104947333
2   2014          None   615792857
3   2017          None   465448852
4   2007          None   662430963
5   2016          None   603322301
6   2020          None   262362411
7   2021          None   295469952
8   2010          None   651457191
9   2022          None   296642507
10  2006          None   547972991
11  2015          None   582695183
12  2018          None   329711260
13  2009          None   759139403
14  2011          None   651936894
15  2008          None   752001864
16  2013          None   575391138
17  2012          None   615799526


C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_5068\1650763272.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  provDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_provider_type_counts;', con=connection)


In [69]:
provAntiJoinQuery = '''
    SELECT a.*
    FROM qa_reporting.dw_iqva_provider_type_counts a
    LEFT JOIN reference_tables.ref_provider_specialty b ON a.provider_type = b.provider_specialty_cd
    WHERE b.provider_specialty_cd IS NULL;
'''

invalidProvDf = pd.read_sql(provAntiJoinQuery, con=connection)
invalidProvDf

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_5068\2573287245.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  invalidProvDf = pd.read_sql(provAntiJoinQuery, con=connection)


,year,provider_type,type_count
0,2019,None,333397785
1,2023,None,104947333
2,2014,None,615792857
3,2017,None,465448852
4,2007,None,662430963
5,2016,None,603322301
6,2020,None,262362411
7,2021,None,295469952
8,2010,None,651457191
9,2013,None,575391138
